###**Data Collection**

Data availed from below websites:

Fot TMDB, data scraping was done to get more fields for analysis


*   https://www.themoviedb.org/
*   https://www.imdb.com/

#### TMDB



In [134]:
import pandas as pd
import ast
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import ttest_1samp
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# SCRAPING THE TMDB WEBSITE FOR DATA USING API CALLS
file_path = '/content/drive/MyDrive/tmdb.movies.csv.gz'
data = pd.read_csv(file_path)

# TMDB API Key
API_KEY = '1747bacc4f65a1c1db0f28b00f7ec810'
BASE_URL = 'https://api.themoviedb.org/3'

# Function to query TMDB API
def fetch_movie_details(movie_id):
    try:
        # Movie details endpoint
        movie_url = f"{BASE_URL}/movie/{movie_id}?api_key={API_KEY}"
        credits_url = f"{BASE_URL}/movie/{movie_id}/credits?api_key={API_KEY}"

        # Fetch movie details
        movie_response = requests.get(movie_url).json()
        credits_response = requests.get(credits_url).json()

        # Extract required data
        overview = movie_response.get('overview', 'No overview available')
        revenue = movie_response.get('revenue', 'N/A')
        cast = [member['name'] for member in credits_response.get('cast', [])[:5]]  # Top 5 cast members
        director = next(
            (crew['name'] for crew in credits_response.get('crew', []) if crew['job'] == 'Director'),
            'N/A'
        )

        return {
            'Overview': overview,
            'Revenue': revenue,
            'Top Cast': ', '.join(cast),
            'Director': director
        }
    except Exception as e:
        return {
            'Overview': 'Error',
            'Revenue': 'Error',
            'Top Cast': 'Error',
            'Director': 'Error',
            'Error': str(e)
        }

# Add new columns for enriched data
data['Overview'] = None
data['Revenue'] = None
data['Top Cast'] = None
data['Director'] = None

# Fetch data for each movie
for index, row in data.iterrows():
    movie_id = row['id']
    details = fetch_movie_details(movie_id)
    data.at[index, 'Overview'] = details['Overview']
    data.at[index, 'Revenue'] = details['Revenue']
    data.at[index, 'Top Cast'] = details['Top Cast']
    data.at[index, 'Director'] = details['Director']

    print(f"Processed movie ID {movie_id}")  # Progress update
    time.sleep(0.2)  # Respect API rate limits (adjust as needed)

# Save the enriched dataset
output_file = 'enriched_tmdb_movies.csv'
data.to_csv(output_file, index=False)
print(f"Enriched data saved to {output_file}")


Streaming output truncated to the last 5000 lines.
Processed movie ID 58760
Processed movie ID 122372
Processed movie ID 70862
Processed movie ID 77812
Processed movie ID 61400
Processed movie ID 83125
Processed movie ID 82929
Processed movie ID 57941
Processed movie ID 50272
Processed movie ID 109581
Processed movie ID 78309
Processed movie ID 76815
Processed movie ID 108251
Processed movie ID 65953
Processed movie ID 60421
Processed movie ID 63686
Processed movie ID 57889
Processed movie ID 68335
Processed movie ID 87108
Processed movie ID 296192
Processed movie ID 286328
Processed movie ID 82756
Processed movie ID 109230
Processed movie ID 79201
Processed movie ID 83891
Processed movie ID 118249
Processed movie ID 87820
Processed movie ID 45075
Processed movie ID 48573
Processed movie ID 77261
Processed movie ID 67943
Processed movie ID 55151
Processed movie ID 91930
Processed movie ID 120370
Processed movie ID 204973
Processed movie ID 80201
Processed movie ID 75507
Processed movie

In [184]:
# MAPPING THE MOVIES TO THEIR RESPECTIVE GENRES USING GENRE IDS
 # To evaluate genre_ids strings as Python lists

# Load the enriched dataset
file_path = 'enriched_tmdb_movies.csv'
data = pd.read_csv(file_path)

# Predefined TMDB Genre Mapping
genre_mapping = {
    28: "Action",
    12: "Adventure",
    16: "Animation",
    35: "Comedy",
    80: "Crime",
    99: "Documentary",
    18: "Drama",
    10751: "Family",
    14: "Fantasy",
    36: "History",
    27: "Horror",
    10402: "Music",
    9648: "Mystery",
    10749: "Romance",
    878: "Science Fiction",
    10770: "TV Movie",
    53: "Thriller",
    10752: "War",
    37: "Western"
}

# Function to map genre IDs to names
def map_genres(genre_ids_str):
    try:
        genre_ids = ast.literal_eval(genre_ids_str)  # Convert string to list
        return ', '.join([genre_mapping.get(genre_id, "Unknown") for genre_id in genre_ids])
    except Exception as e:
        return "Unknown"

# Apply the mapping to the genre_ids column
data['Genres'] = data['genre_ids'].apply(map_genres)

# Save the updated dataset
output_file = 'final_tmdb.xlsx'
data.to_excel(output_file, index=False)

print(f"Updated dataset with genres saved to {output_file}")


Updated dataset with genres saved to final_tmdb.xlsx


In [185]:
tmdb_data = pd.read_excel('final_tmdb.xlsx')

In [187]:
# Merge with the tmdb budget dataset on 'id'
budget_data = pd.read_csv('budget_data.csv')
tmdb_data = pd.merge(tmdb_data, budget_data, on='id', how='left')

In [188]:
tmdb_data.head()

Unnamed: 0            genre_ids     id original_language  \
0           0      [12, 14, 10751]  12444                en   
1           1  [14, 12, 16, 10751]  10191                en   
2           2        [12, 28, 878]  10138                en   
3           3      [16, 35, 10751]    862                en   
4           3      [16, 35, 10751]    862                en   

                                 original_title  popularity release_date  \
0  Harry Potter and the Deathly Hallows: Part 1      33.533   2010-11-19   
1                      How to Train Your Dragon      28.734   2010-03-26   
2                                    Iron Man 2      28.515   2010-05-07   
3                                     Toy Story      28.005   1995-11-22   
4                                     Toy Story      28.005   1995-11-22   

                                          title  vote_average  vote_count  \
0  Harry Potter and the Deathly Hallows: Part 1           7.7       10788   
1                      How to Train Your Dragon           7.7        7610   
2                                    Iron Man 2           6.8       12368   
3                                     Toy Story           7.9       10174   
4                                     Toy Story           7.9       10174   

                                            Overview    Revenue  \
0  Harry, Ron and Hermione walk away from their l...  954305868   
1  As the son of a Viking leader on the cusp of m...  494879471   
2  With the world now aware of his dual life as t...  623933331   
3  Led by Woody, Andy's toys live happily in his ...  394436586   
4  Led by Woody, Andy's toys live happily in his ...  394436586   

                                            Top Cast       Director  \
0  Daniel Radcliffe, Emma Watson, Rupert Grint, T...    David Yates   
1  Jay Baruchel, Gerard Butler, Craig Ferguson, A...   Dean DeBlois   
2  Robert Downey Jr., Gwyneth Paltrow, Don Cheadl...    Jon Favreau   
3  Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...  John Lasseter   
4  Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...  John Lasseter   

                                  Genres       Budget  
0             Adventure, Fantasy, Family  250000000.0  
1  Fantasy, Adventure, Animation, Family  165000000.0  
2     Adventure, Action, Science Fiction  200000000.0  
3              Animation, Comedy, Family   30000000.0  
4              Animation, Comedy, Family   30000000.0

In [190]:
tmdb_data.shape

(28605, 16)

#### IMDB

In [180]:
file = "im.db"
con =sqlite3.connect(file)

In [181]:
cur = con.cursor()

In [182]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()
print(tables)

[('movie_basics',), ('directors',), ('known_for',), ('movie_akas',), ('movie_ratings',), ('persons',), ('principals',), ('writers',)]


In [183]:
df = pd.read_sql("SELECT * FROM movie_basics", con)
df.head()

movie_id                    primary_title              original_title  \
0  tt0063540                        Sunghursh                   Sunghursh   
1  tt0066787  One Day Before the Rainy Season             Ashad Ka Ek Din   
2  tt0069049       The Other Side of the Wind  The Other Side of the Wind   
3  tt0069204                  Sabse Bada Sukh             Sabse Bada Sukh   
4  tt0100275         The Wandering Soap Opera       La Telenovela Errante   

   start_year  runtime_minutes                genres  
0        2013            175.0    Action,Crime,Drama  
1        2019            114.0       Biography,Drama  
2        2018            122.0                 Drama  
3        2018              NaN          Comedy,Drama  
4        2017             80.0  Comedy,Drama,Fantasy

In [ ]:
df2 = pd.read_sql("SELECT * FROM movie_akas", con)
df2.head()

movie_id  ordering                                    title region  \
0  tt0369610        10                            Джурасик свят     BG   
1  tt0369610        11                        Jurashikku warudo     JP   
2  tt0369610        12  Jurassic World: O Mundo dos Dinossauros     BR   
3  tt0369610        13                  O Mundo dos Dinossauros     BR   
4  tt0369610        14                           Jurassic World     FR   

  language        types   attributes  is_original_title  
0       bg         None         None                0.0  
1     None  imdbDisplay         None                0.0  
2     None  imdbDisplay         None                0.0  
3     None         None  short title                0.0  
4     None  imdbDisplay         None                0.0

In [ ]:
df.shape

(146144, 6)

##**DATA PREPARATION**

This will be handled through:

*   Combining tables from different databases
*   Handling Duplicates: Ensuring there are no repeated records
*   Handling Missing Data:Fill in or remove incomplete data points, such as missing report status.
*   Exploratory Data Analysis (EDA)


#### **MERGING TMDB AND IMDB DATA**

In [233]:
# Load the TMDb and IMDb data
imdb_data = df

# Normalize title columns by converting to lowercase and trimming whitespace
tmdb_data['title_normalized'] = tmdb_data['title'].str.lower().str.strip()
imdb_data['primary_title_normalized'] = imdb_data['primary_title'].str.lower().str.strip()

# Check for common titles
common_titles = set(tmdb_data['title_normalized']).intersection(set(imdb_data['primary_title_normalized']))

# Count the number of movies in both datasets
common_count = len(common_titles)
print(f"Number of common movies between TMDb and IMDb based on titles: {common_count}")

Number of common movies between TMDb and IMDb based on titles: 15369


In [234]:
tmdb_common = tmdb_data[tmdb_data['title_normalized'].isin(common_titles)].sort_values(by='title')
imdb_common = imdb_data[imdb_data['primary_title_normalized'].isin(common_titles)].sort_values(by='primary_title')

In [235]:
# Load the TMDb and IMDb data and remove duplicated
tmdb_data = tmdb_common
imdb_data = imdb_common
tmdb_data = tmdb_data.drop_duplicates(subset=['title_normalized'])
imdb_data = imdb_data.drop_duplicates(subset=['primary_title_normalized'])

# Normalizing title columns by converting to lowercase and trimming whitespace
tmdb_data['title_normalized'] = tmdb_data['title'].str.lower().str.strip()
imdb_data['primary_title_normalized'] = imdb_data['primary_title'].str.lower().str.strip()

# Inner join on normalized titles
merged_data = pd.merge(
    imdb_data[['primary_title', 'start_year', 'runtime_minutes', 'genres', 'primary_title_normalized']],
    tmdb_data[['title', 'vote_average', 'vote_count', 'Revenue', 'Top Cast', 'Director', 'Genres', 'title_normalized', 'Budget','popularity']],
    left_on='primary_title_normalized',
    right_on='title_normalized',
    how='inner'
)

# Desired columns in the result
merged_data = merged_data[[
    'primary_title', 'start_year', 'runtime_minutes', 'genres', 'vote_average',
    'vote_count', 'Top Cast', 'Director', 'popularity', 'Budget', 'Revenue'
]]

# Save the merged dataset to a file
merged_data.to_csv('merged_movies.csv', index=False)

print("Merged dataset saved successfully!")


<ipython-input-235-1831bf5d6eac>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_data['primary_title_normalized'] = imdb_data['primary_title'].str.lower().str.strip()


Merged dataset saved successfully!
